In [ ]:
ex_text = input('작성하고자 하는 뉴스의 카테고리를 입력해주세요 : ')
print("\n" + '작성하시고자 하는 카테고리가 다음과 같습니까? ' + ex_text)
print("\n" + '맞으면 숫자 1, 아니면 2를 눌러주세요')
input_number = input()

if input_number == 1:
  ex_text = ex_text
  print(ex_text + '로 선택되었습니다.')
elif input_number == 2:
  ex_text = input('작성하고자 하는 뉴스의 카테고리를 입력해주세요 : ')
  ex_text = ex_text

작성하고자 하는 뉴스의 카테고리를 입력해주세요 : 취업

작성하시고자 하는 카테고리가 다음과 같습니까? 취업

맞으면 숫자 1, 아니면 2를 눌러주세요
1


In [ ]:
ex_text

'스포츠'

In [ ]:
from urllib import request, parse
from pprint import pprint
import json
headers = {'Content-Type': 'application/json; chearset=utf-8'}
data = {
    'access_key': 'c623a12a-2a15-44bc-8d93-183b5b6834b4',
    'argument': {
        'query' : '취업',
        'published_at': {
            'from' : '2022-08-01',
            'until' : '2022-08-31'
        },
        'return_from' : 0,
        'return_size' : 1000,
        "fields" : ['content']
        }
        }

url = 'http://tools.kinds.or.kr:8888/search/news'
req = request.Request(url, headers=headers, data=json.dumps(data).encode('utf-8'))
res = request.urlopen(req)


json_str = res.read().decode('utf-8')
json_file = json.loads(json_str)
json_file[:1]

In [ ]:
#get the data
data_list = []
for i in range(0, 1000):
  x = json_file['return_object']['documents'][i]['content']
  data_list.append(x)

In [ ]:
#data preprocessing
import re
data_preprocessing = []
for i in range(len(data_list)):
  k = str(data_list[i].replace('\n', '')) # replace 활용
  k = re.sub('[a-zA-Z]', '', k) # re package 사용
  k = re.sub('@', '', k)
  k = re.sub("\xa0", '', k)
  k = re.sub("’", '', k)
  k = re.sub('“', '', k)
  k = re.sub("'", '', k)
  k = re.sub("‘", '', k)
  k = re.sub('”', '', k)
  k = re.sub('[^\w\s]', '', k)
  data_preprocessing.append(k)

In [ ]:
pip install kobert-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 2.1 MB/s 
     |████████████████████████████████| 1.3 MB 34.7 MB/s 
     |████████████████████████████████| 6.6 MB 45.9 MB/s 
     |████████████████████████████████| 120 kB 68.2 MB/s 


In [ ]:
#data tokenization by kobert
from kobert_transformers import get_tokenizer

tokenizer = get_tokenizer()
data_kobert_token = []
for i in range(0, 1000):
  k = tokenizer.tokenize("[CLS]" + data_preprocessing[i] + "[SEP]")
  j = tokenizer.convert_tokens_to_ids(k) 
  data_kobert_token.append(j)

#check vocab size
vocab_size = tokenizer.vocab_size
print(tokenizer.vocab_size)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


8002


In [ ]:
#check the length of sequences

print('max_length : {}'.format(max(len(i) for i in data_kobert_token)))
print('average : {}'.format(sum(map(len, data_kobert_token))/len(data_kobert_token)))

max_length : 1972
average : 385.797


In [ ]:
#padding(pre)
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense
max_length = 512
data_padding = pad_sequences(data_kobert_token, maxlen = max_length)

In [ ]:
#build Attention 
import tensorflow as tf

class Att(tf.keras.Model):
  def __init__(self, units):
    super(Att, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, query, values):
    # query hidden state -> (batch_size, hidden size)
    # query_with_time_axis -> (batch_size, 1, hidden size)
    # values -> (batch_size, max_len, hidden size)
    # to get score, have to plus, so, expand time series vector
    query_with_time_axis = tf.expand_dims(query, 1)

    # score -> (batch_size, max_length, 1)
    # score get only  one value sclar because of self.V layer
    # self.V -> (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights -> (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    #context_vector -> (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights



In [ ]:
#prepare dataset by tensorflow
#setting the hyper prameters
buffer_size = len(data_padding) # shuffle space 
batch_size= 1000
# steps_per_epoch = len(data_padding)//batch_size
embedding_dim = 256
units = 1024

#tensor_slices -> traning, target but here, we don't have target data
dataset = tf.data.Dataset.from_tensor_slices(data_padding).shuffle(buffer_size)
dataset = dataset.batch(batch_size, drop_remainder=True) # drop_remainder -> last data drop or not?

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, units, batch_size):
    super(Encoder, self).__init__()
    self.batch_size = batch_size
    self.encoder_units = units
    self.embedding_layer = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.GRU = tf.keras.layers.GRU(self.encoder_units,
                                     return_sequences = True,
                                     return_state = True,
                                     recurrent_initializer = 'glorot_uniform')

  def call(self, data, hidden):
    data =  self.embedding_layer(data)
    output, state = self.GRU(data, initial_state = hidden)
    return output, state

  def hidden_state(self):
    return tf.zeros((self.batch_size, self.encoder_units))

In [ ]:
#next and iter function
data_input = next(iter(dataset))
print(data_input.shape)

(1000, 512)


In [ ]:
encoder = Encoder(vocab_size, embedding_dim, units, batch_size)

hidden_layer = encoder.hidden_state()
layer_output, hidden_layer = encoder(data_input, hidden_layer)

print(hidden_layer)

tf.Tensor(
[[-0.00243207  0.00319222  0.01027524 ... -0.00347601 -0.00044311
   0.00791706]
 [-0.0014424  -0.0007148   0.01051171 ... -0.00060891 -0.00206686
   0.01002387]
 [-0.0030315   0.00226851  0.01247618 ... -0.00054188 -0.00011752
   0.01342034]
 ...
 [-0.01068841 -0.0019068   0.00577435 ...  0.00108308 -0.00619052
   0.00943835]
 [-0.00539485 -0.0014702   0.01128644 ...  0.00462034  0.00330396
   0.01416178]
 [-0.00117034  0.00232054  0.01129279 ... -0.0010222  -0.00092475
   0.01183183]], shape=(1000, 1024), dtype=float32)


In [ ]:
attention_layer = Att(16)
context_vector, weights = attention_layer(hidden_layer, layer_output)

In [ ]:
#Decoder

class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, decoder_units, batch_size):
    super(Decoder, self).__init__()
    self.batch_size = batch_size
    self.decoder_units = decoder_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.decoder_units,
                                   return_sequences = True,
                                   return_state = True,
                                   recurrent_initializer = 'glorot_uniform')
    self.fullyconnect = tf.keras.layers.Dense(vocab_size)

    self.attention = Att(self.decoder_units)

  def call(self, data, hidden, encoder_output):
    context_vector, attention_weights = self.attention(hidden, encoder_output)
    data = self.embedding(data)
    data = tf.concat([tf.expand_dims(context_vector, 1), data], axis = -1)
    output, state = self.gru(data)
    output = tf.reshape(output, (-1, output.shape[2]))
    data = self.fullyconnect(output)

    return data, state, attention_weights

In [ ]:
decoder = Decoder(vocab_size, embedding_dim, units, batch_size)

decoder_output, _, _ = decoder(tf.random.uniform((batch_size, 1)),
                               hidden_layer, layer_output) 

In [ ]:
decoder_output[1]

<tf.Tensor: shape=(8002,), dtype=float32, numpy=
array([ 0.00045828,  0.002942  , -0.00103849, ..., -0.00237093,
       -0.00137342,  0.00231997], dtype=float32)>